# LangChain Basics

In [1]:
%pip install -r ./requirements.txt --use-deprecated=legacy-resolver -q

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip show langchain

Name: langchain
Version: 0.1.10
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: c:\users\ashwi\anaconda3\lib\site-packages
Requires: aiohttp, async-timeout, dataclasses-json, jsonpatch, langchain-community, langchain-core, langchain-text-splitters, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: langchain-experimental
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip show OpenAI

Name: openai
Version: 1.13.3
Summary: The official Python library for the openai API
Home-page: 
Author: 
Author-email: OpenAI <support@openai.com>
License: 
Location: c:\users\ashwi\anaconda3\lib\site-packages
Requires: anyio, distro, httpx, pydantic, sniffio, tqdm, typing-extensions
Required-by: langchain-openai
Note: you may need to restart the kernel to use updated packages.


In [4]:
#pip install langchain --upgrade -q

### Python-dotenv

In [5]:
import os
from dotenv import load_dotenv, find_dotenv
#load_dotenv('./.env')
load_dotenv(find_dotenv(), override = True)

os.environ.get('PINECONE_API_KEY')
#os.environ.get('OPENAI_API_KEY')

'e10f7206-10b5-474b-a082-dd1aef446c82'

# ChatModels :  GPT-3.5-Turbo and GPT-4

In [6]:
from langchain.schema import(
    AIMessage,
    HumanMessage,
    SystemMessage
)
from langchain.chat_models import ChatOpenAI

In [7]:
chat  = ChatOpenAI(model_name ='gpt-3.5-turbo', temperature =0.5, max_tokens=1024)
messages = [
    SystemMessage(content = 'You are physicist an respond only in German'),
    HumanMessage(content= 'explain quantum mechanis in one sentence')
]
output = chat(messages)

C:\Users\ashwi\anaconda3\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
C:\Users\ashwi\anaconda3\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


In [8]:
print(output.content)

Quantenmechanik beschreibt das Verhalten von Teilchen auf atomarer und subatomarer Ebene durch Wellenfunktionen und Wahrscheinlichkeitsverteilungen.


# Caching LLM Responses

##1. In-Memory Cache

In [9]:
#%pip install OpenAI

In [10]:
from langchain.globals import set_llm_cache
from langchain_openai import OpenAI
llm = OpenAI(model_name='gpt-3.5-turbo-instruct')

In [11]:
from langchain.cache import InMemoryCache
set_llm_cache(InMemoryCache())

In [12]:
%%time
prompt = 'Tell a me a joke that a toddler can understand.'
llm.invoke(prompt)

Wall time: 1.06 s


'\n\nWhy did the tomato turn red?\n\nBecause it saw the salad dressing!'

In [13]:
%%time
llm.invoke(prompt)

Wall time: 1.31 ms


'\n\nWhy did the tomato turn red?\n\nBecause it saw the salad dressing!'

### 2. SQLite Caching

In [15]:
from langchain.cache import SQLiteCache
set_llm_cache(SQLiteCache(database_path=".langchain.db"))

In [16]:
%%time
# First request (not in cache, takes longer)
llm.invoke("Tell me a joke")

Wall time: 982 ms


"\n\nWhy couldn't the bicycle stand up by itself? Because it was two-tired."

In [17]:
%%time
# Second request (cached, faster)
llm.invoke("Tell me a joke")

Wall time: 48.3 ms


"\n\nWhy couldn't the bicycle stand up by itself? Because it was two-tired."

## LLM Streaming

In [18]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()
prompt = 'Write a rock song about the Moon and a Raven.'
print(llm.invoke(prompt).content)

Verse 1:
In the dead of night, under the silver moon
A raven flies, singing a haunting tune
Its wings beat against the darkened sky
As it calls out to the moon up high

Chorus:
Oh moon and raven, in the night so still
Your spirits dance, in the darkness they fulfill
A bond unbroken, a connection so true
Moon and raven, forever in view

Verse 2:
The moon shines bright, guiding the raven's flight
Through the shadows and the fading light
The raven's cry echoes through the night
A symbol of mystery, a symbol of might

Chorus:
Oh moon and raven, in the night so still
Your spirits dance, in the darkness they fulfill
A bond unbroken, a connection so true
Moon and raven, forever in view

Bridge:
The moon's pale face, a beacon in the sky
The raven's black wings, soaring high
Together they roam, through the endless night
A duo of darkness, a duo of light

Chorus:
Oh moon and raven, in the night so still
Your spirits dance, in the darkness they fulfill
A bond unbroken, a connection so true
Moon a

In [19]:
for chunk in llm.stream(prompt):
    print(chunk.content, end='', flush=True)

Verse 1:
Underneath the midnight sky
The moon shines bright up high
A raven calls out in the night
A haunting melody, a beautiful sight

Chorus:
Moonlight shining, raven's cry
A mysterious dance in the sky
The moon and the raven, a timeless bond
In the darkness, a magic beyond

Verse 2:
The raven soars, the moon above
Two creatures connected by love
In the stillness of the night
Their spirits take flight

Chorus:
Moonlight shining, raven's cry
A mysterious dance in the sky
The moon and the raven, a timeless bond
In the darkness, a magic beyond

Bridge:
Together they roam
In the night they find their home
A connection so deep
In the darkness they keep

Chorus:
Moonlight shining, raven's cry
A mysterious dance in the sky
The moon and the raven, a timeless bond
In the darkness, a magic beyond

Outro:
As the night fades away
The moon and the raven stay
A bond unbroken, a love so true
In the darkness, they'll always be together, me and you.

## PromptTemplates

In [20]:
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# Define a template for the prompt
template = '''You are an experienced virologist.
Write a few sentences about the following virus "{virus}" in {language}.'''

# Create a PromptTemplate object from the template
prompt_template = PromptTemplate.from_template(template=template)

# Fill in the variable: virus and language
prompt = prompt_template.format(virus='hiv', language='german')
prompt  # Returns the generated prompt


'You are an experienced virologist.\nWrite a few sentences about the following virus "hiv" in german.'

In [21]:
llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0)
output = llm.invoke(prompt)
print(output.content)

HIV, das humane Immundefizienzvirus, ist ein Retrovirus, das das menschliche Immunsystem schwächt und zu AIDS führen kann. Es wird hauptsächlich durch den Kontakt mit infiziertem Blut, sexuellen Kontakt oder von der Mutter auf das Kind während der Schwangerschaft übertragen. Es gibt keine Heilung für HIV, aber mit antiretroviralen Medikamenten kann die Krankheit kontrolliert und das Leben der Betroffenen verlängert werden. Die Prävention von HIV-Infektionen durch sicheren Sex und den Austausch von Nadeln ist entscheidend im Kampf gegen die Ausbreitung des Virus.


## ChatPromptTemplates

In [22]:
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage

# Create a chat template with system and human messages
chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content='You respond only in the JSON format.'),
        HumanMessagePromptTemplate.from_template('Top {n} countries in {area} by population.')
    ]
)

# Fill in the specific values for n and area
messages = chat_template.format_messages(n='5', area='World')
print(messages)  # Outputs the formatted chat messages


[SystemMessage(content='You respond only in the JSON format.'), HumanMessage(content='Top 5 countries in World by population.')]


In [23]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI()
output = llm.invoke(messages)
print(output.content)

{
    "countries": [
        {
            "rank": 1,
            "name": "China",
            "population": "1,439,323,776"
        },
        {
            "rank": 2,
            "name": "India",
            "population": "1,380,004,385"
        },
        {
            "rank": 3,
            "name": "United States",
            "population": "331,002,651"
        },
        {
            "rank": 4,
            "name": "Indonesia",
            "population": "273,523,615"
        },
        {
            "rank": 5,
            "name": "Pakistan",
            "population": "220,892,340"
        }
    ]
}


## Simple Chains

In [24]:
from langchain_openai import ChatOpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain

llm = ChatOpenAI()
template = '''You are an experience virologist.
Write a few sentences about the following virus "{virus}" in {language}.'''
prompt_template = PromptTemplate.from_template(template=template)

chain = LLMChain(
    llm=llm,
    prompt=prompt_template,
    verbose=True
)

output = chain.invoke({'virus': 'HSV', 'language': 'Spanish'})




> Entering new LLMChain chain...
Prompt after formatting:
You are an experience virologist.
Write a few sentences about the following virus "HSV" in Spanish.

> Finished chain.


In [25]:
print(output)

{'virus': 'HSV', 'language': 'Spanish', 'text': 'El virus del herpes simple (HSV) es un virus de la familia de los herpesviridae que puede causar infecciones en humanos. Se transmite principalmente a través del contacto directo con heridas o lesiones infectadas. Existen dos tipos de HSV: el HSV-1, que suele causar herpes labial, y el HSV-2, que es responsable del herpes genital. Aunque no tiene cura, existen tratamientos antivirales que pueden ayudar a controlar los síntomas y prevenir brotes recurrentes.'}


In [26]:
template = 'What is the capital of {country}?. List the top 3 places to visit in that city. Use bullet points'
prompt_template = PromptTemplate.from_template(template=template)

# Initialize an LLMChain with the ChatOpenAI model and the prompt template
chain = LLMChain(
    llm=llm,
    prompt=prompt_template,
    verbose=True
)

country = input('Enter Country: ')

# Invoke the chain with specific virus and language values
output = chain.invoke(country)
print(output['text'])   

Enter Country: SPain


> Entering new LLMChain chain...
Prompt after formatting:
What is the capital of SPain?. List the top 3 places to visit in that city. Use bullet points

> Finished chain.
The capital of Spain is Madrid. 

Top 3 places to visit in Madrid:

- The Prado Museum
- Retiro Park
- Royal Palace of Madrid


## Sequential Chains

In [29]:
from langchain_openai import ChatOpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain

# Initialize the first ChatOpenAI model (gpt-3.5-turbo) with specific temperature
llm1 = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0.5)

# Define the first prompt template
prompt_template1 = PromptTemplate.from_template(
    template='You are an experienced scientist and Python programmer. Write a function that implements the concept of {concept}.'
)
# Create an LLMChain using the first model and the prompt template
chain1 = LLMChain(llm=llm1, prompt=prompt_template1)

# Initialize the second ChatOpenAI model (gpt-4-turbo) with specific temperature
llm2 = ChatOpenAI(model_name='gpt-4-turbo-preview', temperature=1.2)

# Define the second prompt template
prompt_template2 = PromptTemplate.from_template(
    template='Given the Python function {function}, describe it as detailed as possible.'
)
# Create another LLMChain using the second model and the prompt template
chain2 = LLMChain(llm=llm2, prompt=prompt_template2)

# Combine both chains into a SimpleSequentialChain
overall_chain = SimpleSequentialChain(chains=[chain1, chain2], verbose=True)

# Invoke the overall chain with the concept "linear regression"
output = overall_chain.invoke('linear regression')




> Entering new SimpleSequentialChain chain...
Sure! Here is a simple implementation of linear regression in Python:

```python
import numpy as np

def linear_regression(x, y):
    n = len(x)
    x_mean = np.mean(x)
    y_mean = np.mean(y)

    numerator = np.sum((x - x_mean) * (y - y_mean))
    denominator = np.sum((x - x_mean) ** 2)

    slope = numerator / denominator
    intercept = y_mean - slope * x_mean

    return slope, intercept

# Example usage
x = np.array([1, 2, 3, 4, 5])
y = np.array([2, 4, 5, 4, 5])

slope, intercept = linear_regression(x, y)
print("Slope:", slope)
print("Intercept:", intercept)
```

This function takes two arrays `x` and `y` as input, calculates the slope and intercept of the linear regression line that best fits the data, and returns them. You can then use these values to predict the y-values for new x-values or visualize the regression line.


NotFoundError: Error code: 404 - {'error': {'message': 'The model `gpt-4-turbo-preview` does not exist or you do not have access to it. Learn more: https://help.openai.com/en/articles/7102672-how-can-i-access-gpt-4.', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_found'}}

In [30]:
print(output['output'])

KeyError: 'output'

In [31]:
pip install -q langchain_experimental

Note: you may need to restart the kernel to use updated packages.


In [32]:
from langchain_experimental.utilities import PythonREPL
python_repl = PythonREPL()
python_repl.run('print([n for n in range(1, 100) if n % 13 == 0])')

Python REPL can execute arbitrary code. Use with caution.


'[13, 26, 39, 52, 65, 78, 91]\n'

In [33]:
from langchain_experimental.agents.agent_toolkits import create_python_agent
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain_openai import ChatOpenAI

# Initialize the ChatOpenAI model with gpt-4-turbo and a temperature of 0
llm = ChatOpenAI(model='gpt-4-turbo-preview', temperature=0)

# Create a Python agent using the ChatOpenAI model and a PythonREPLTool
agent_executor = create_python_agent(
    llm=llm,
    tool=PythonREPLTool(),
    verbose=True
)

# Invoke the agent
prompt = 'Calculate the square root of the factorial of 12 and display it with 4 decimal points'
agent_executor.invoke(prompt)



> Entering new AgentExecutor chain...


NotFoundError: Error code: 404 - {'error': {'message': 'The model `gpt-4-turbo-preview` does not exist or you do not have access to it. Learn more: https://help.openai.com/en/articles/7102672-how-can-i-access-gpt-4.', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_found'}}

In [34]:
response = agent_executor.invoke('What is the answer to 5.1 ** 7.3?')



> Entering new AgentExecutor chain...


NotFoundError: Error code: 404 - {'error': {'message': 'The model `gpt-4-turbo-preview` does not exist or you do not have access to it. Learn more: https://help.openai.com/en/articles/7102672-how-can-i-access-gpt-4.', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_found'}}

In [35]:
response

NameError: name 'response' is not defined

In [36]:
print(response['input'])

NameError: name 'response' is not defined

In [37]:
print(response['output'])


NameError: name 'response' is not defined